<h2>今回の方針と結果</h2><p>
    
複数の識別器で学習し、アンサンブルで予測値を求める
<ul>
    <li>ロジスティック回帰、サポートベクターマシン、ランダムフォレスト、ナイーブベイズ</li>
</ul>
その前に、データの整形や白色化などの処理を実施する
<p>
でもあまり精度上がらず…　w<br>
アンサンブルの前に、各識別器の精度向上がもう少し必要。<br>

識別機の精度向上のために試した方策：
<ul>
    <li>説明変数に二乗の項を追加する：効果あり。例えばロジスティック回帰で、正答率が数%pt程度上昇。過少適合状態の改善？</li>
    <li>学習データのサイズを増減させる：効果不明瞭。学習データを増やすと正答率が悪くなることもあった</li>
    <li>各識別器のパラメータを調整する：効果不明瞭。本当はグリッドサーチなどでもう少し系統的にやった方がよい</li>
</ul>
<p>
ロジスティック回帰、サポートベクターマシンおよびナイーブベイズは、訓練性能、汎化性能ともに満足いくものではなく、おそらく過少適合状態。学習データを増やしてもその状況は大きく変化なし。<p>
    
ランダムフォレストは階層を増やす(max_depth = None (default))と訓練性能は非常に高くなった（正答率 ~96%）ものの、汎化性能とのギャップが大きく過学習状態になっていることが見て取れる。<p>

なお、アンサンブル学習により精度があまり上がらないのは、各識別器の意見（予測値）があまり割れていないからだと考えられる。（間違えるときはみんな同じように間違える）

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.externals.six import StringIO
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

import graphviz
import pydotplus
from IPython.display import Image

C:\Users\05HWA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv("./ks-projects-201801.csv")
display((df).head())

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
#ファンディングの募集期間 Durationを計算
df["deadline"] = pd.to_datetime(df['deadline'])
df["launched"] = pd.to_datetime(df['launched'])
df["duration"] = df["deadline"] - df["launched"]

#Durationを日数に変換し、一日当たりの目標金額を求める
df['duration_days'] = df['duration'].dt.total_seconds() / (24 * 60 * 60)
df['target_per_day'] = df['usd_goal_real'] / df['duration_days']

In [4]:
#入力データの整形

df1 = df.query('state == ["successful", "failed", "canceled"]')  # stateを成功、失敗、キャンセルだけに絞る

df1 = df1[df1['duration_days'] < 1000]   # 外れ値を削除。募集期間1000日以上は無効データと判断
df1 = df1[df1['target_per_day'] < 1e6]   # 同上。目標金額が一日当たり100万ドル以上の場合は無効データと判断

df1["success"] = df1["state"] == "successful"  # ファンディングの結果が成功かそうでないかで2クラスに分類

In [5]:
# カテゴリ変数のうち、インスタンス数の少ない値はothersにまとめる
df1.country = df1.country.replace(["AT", "BE", "CH", "DK", "HK", "IE", "LU", "NO", "SG", "JP"], "others")
df1.main_category = df1.main_category.replace(["Comics", "Crafts", "Dance", "Journalism", "Photography"], "others")
df1.currency = df1.currency.replace(["CHF", "HKD", "JPY", "NOK", "SGD"], "others")

In [6]:
# カテゴリ変数からone-hotベクトルを生成
A = pd.get_dummies(df1["main_category"],drop_first = True)
B = pd.get_dummies(df1["country"], drop_first = True)
C = pd.get_dummies(df1["currency"], drop_first = True)

# 説明変数のベクトルを作成
alpha = df1["usd_goal_real"]
beta = df1["target_per_day"]
gamma = df1["duration_days"]

A1 = pd.concat([A, B, C, alpha, beta, gamma, alpha **2, beta ** 2, gamma **2], axis = 1).values

In [7]:
#説明変数の白色化

#無相関化
cov = np.cov(A1, rowvar=0)
v, S = np.linalg.eig(cov)
sdata = np.dot(S.T, A1.T).T

#標準化
sc = StandardScaler()
sc.fit(sdata)
A2 = sc.transform(sdata)

In [47]:
# 訓練データ、テストデータの切り出し
y = df1["success"].values
X = A2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.95, random_state=0)
print(len(X_train))
print(len(X_test))

18519
351868


In [57]:
# ロジスティック回帰
clf1 = SGDClassifier(loss='log')
clf1.fit(X_train, y_train)
y_est1 = clf1.predict(X_train)

C:\Users\05HWA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [58]:
print('ロジスティック回帰')
print()
print('訓練データ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est1)))
conf_mat1 = pd.DataFrame(confusion_matrix(y_train, y_est1))
display(conf_mat1)

# テストデータによる検証
y_test1 = clf1.predict(X_test) 
print('テストデータ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_test1)))

ロジスティック回帰

訓練データ　正答率 = 62.514%


,0,1
0,9105,2658
1,4284,2472


テストデータ　正答率 = 62.463%


In [59]:
# サポートベクターマシン
C = 10
clf2 = SVC(C=C)

In [60]:
clf2.fit(X_train, y_train)
y_est2 = clf2.predict(X_train) 

print('サポートベクターマシン')
print()
print('訓練データ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est2)))
conf_mat2 = pd.DataFrame(confusion_matrix(y_train, y_est2))
display(conf_mat2)

# テストデータによる検証
y_test2 = clf2.predict(X_test) 
print('テストデータ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_test2)))

サポートベクターマシン

訓練データ　正答率 = 65.803%


,0,1
0,10577,1186
1,5147,1609


テストデータ　正答率 = 65.708%


In [61]:
# ランダムフォレスト
clf3 = RandomForestClassifier()

In [62]:
clf3.fit(X_train, y_train)
y_est3 = clf3.predict(X_train)

print('ランダムフォレスト')
print()
print('訓練データ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est3)))
conf_mat3 = pd.DataFrame(confusion_matrix(y_train, y_est3))
display(conf_mat3)

# テストデータによる検証
y_test3 = clf3.predict(X_test) 
print('ランダムフォレスト　テストデータ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_test3)))

ランダムフォレスト

訓練データ　正答率 = 96.895%


,0,1
0,11678,85
1,490,6266


ランダムフォレスト　テストデータ　正答率 = 62.540%


In [63]:
#ナイーブベイズ（ベルヌーイ分布）
from sklearn.naive_bayes import BernoulliNB
clf4 = BernoulliNB()

In [64]:
clf4.fit(X_train, y_train)
y_est4 = clf4.predict(X_train)

print('ナイーブベイズ')
print()
print('訓練データ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est4)))
conf_mat4 = pd.DataFrame(confusion_matrix(y_train, y_est4))
display(conf_mat4)

# テストデータによる検証
y_test4 = clf0.predict(X_test) 
print('テストデータ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_test4)))

ナイーブベイズ

訓練データ　正答率 = 62.358%


,0,1
0,8693,3070
1,3901,2855


テストデータ　正答率 = 62.895%


In [65]:
#4つの識別器が出力する予測の多数決で予測値を求める

voting=VotingClassifier([('lr', clf1), ('SVM', clf2), ('rf', clf3), ('NB', clf4),], voting = 'hard')
voting.fit(X_train, y_train)

C:\Users\05HWA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


VotingClassifier(estimators=[('lr', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True... warm_start=False)), ('NB', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [66]:
y_est4 = voting.predict(X_train)
print('多数決　訓練データ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_train, y_est4)))

y_test4 = voting.predict(X_test)
print('多数決　テストデータ　正答率 = {:.3f}%'.format(100 * accuracy_score(y_test, y_test4)))

C:\Users\05HWA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


多数決　訓練データ　正答率 = 73.892%
多数決　テストデータ　正答率 = 65.528%


C:\Users\05HWA\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
